In [ ]:
using Revise

In [ ]:
push!(LOAD_PATH, "/home/amir/work/mps/src/")

using KrylovKit
using SparseArrays
using LinearAlgebra
using Plots
using LaTeXStrings

using MatrixProductStateTools
using ExactDiagonalizationTools
using SymTensors
using GaussianFermions
using GutzwillerMPS

In [ ]:
#include("/home/amir/work/mps/test/runtests.jl")

In [ ]:
⊗ = kron
lx = 16
H = xxz_hamiltonian(lx)
es, vs = eigsolve(H, 1, :SR)
extended_sz(lx, l) = sparse(1.0I, 2^(l-1), 2^(l-1)) ⊗ [1.0 0; 0. -1.0] ⊗ sparse(1.0I, 2^(lx-l), 2^(lx-l))
mps = MatrixProductState(lx, 2, vs[1])
sz  = Matrix(sz_half)
szvals = [measure_1point(mps, sz, l) for l in 1:8]
@show szvals
@show norm2(mps)
data1 = measure_2point(mps, sz, sz) 
#data1[[half_measurement_index(lx,1,l+1) for l=1:lx-1]]
plot((2:lx), data1[[half_measurement_index(lx,1,l+1) for l=1:lx-1]], 
    marker=(:circle),xaxis=(L"$x$"), yaxis=(L"$\langle s^z_1 s^z_x\rangle$"), legend=false)

In [ ]:
cm = correlationmatrix(hopping_chain(lx, boundary=:OBC), div(lx,2))
fgs = generate_fishmangates(cm)
mps = fishman2mps(fgs, 100)
data = measure_2point(mps, sz, sz) 
plot((2:lx), real.(data[[half_measurement_index(lx,1,l+1) for l=1:lx-1]]),
    marker=(:circle),xaxis=(L"$x$"), yaxis=(L"$\langle s^z_1 s^z_x\rangle$"), legend=false)

In [ ]:
mpsgutz = zipandgutzwiller!(mps, mps);

In [ ]:
@show mps_dims_are_consistent(mpsgutz)
data = measure_2point(mpsgutz, sz, sz) 
data = real.(data ./ norm2(mpsgutz))
@show norm2(mpsgutz)
p = plot(xaxis=(L"$x$"), yaxis=(L"$\langle s^z_1 s^z_x\rangle$"), legend=false)
plot!(p, [2:lx], data[[half_measurement_index(lx,1,l+1) for l=1:lx-1]], marker=(:circle))
plot!(p, [2:lx], data1[[half_measurement_index(lx,1,l+1) for l=1:lx-1]], marker=(:circle))